*This notebook comes from [A Whirlwind Tour of Python](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp) by Jake VanderPlas (OReilly Media, 2016). This content is licensed [CC0](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/LICENSE). The full notebook listing is available at https://github.com/jakevdp/WhirlwindTourOfPython.*

<!--NAVIGATION-->
< [Defining and Using Functions](08-Defining-Functions.ipynb) | [Contents](Index.ipynb) | [Iterators](10-Iterators.ipynb) >

# 错误和异常

不论你的代码水平怎么样，你终究会在代码中遇到一些错误。
这些错误一般有这三种：

- *语法错误：* 那些由于你写出了不合法的 Python 代码导致的（一般很好修改）
- *运行时错误：* 那些由于句法上合法的代码被执行导致的错误, 也许是由于不合法的用户输入导致的（有时候很好修改）
- *语义错误：* 那些逻辑上的错误，代码执行时并没有出现问题，但是结果却不是你想要的那样（一般很难追踪和修改）

这里我们主要来看一下如何干净地处理那些 *运行时错误*。
我们可以看到，Python 使用它的 *exception handling* 框架来处理运行时错误。

## 运行时错误

如果你写过一些 Python 代码，你应该会遇到一些运行时错误，它们可能发生在各种情况下。

比如，如果你尝试使用一个未定义的变量：

In [3]:
print(Q)

NameError: name 'Q' is not defined

或者你使用一个未定义的操作：

In [4]:
1 + 'abc'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

或者你在尝试计算一个数学上非法的结果：

In [5]:
2 / 0

ZeroDivisionError: division by zero

或者你在尝试访问一个在序列中不存在的元素：

In [6]:
L = [1, 2, 3]
L[1000]

IndexError: list index out of range

注意到在每个例子中，Python 不仅简单地指出了哪里有错误发生，还抛出了一个*有意义*的异常说明，包括了什么东西出错，具体哪一行代码出错。

遇到这样有意义的报错通常在你尝试追踪错误的根源时很有用。

## 捕获异常：``try`` 和 ``except``
Python 给你提供的捕获运行时异常的主要工具就是 ``try``...``except`` 语句，它的基本结构就像这样：

In [7]:
try:
    print("this gets executed first")
except:
    print("this gets executed only if there is an error")

this gets executed first


注意到第二个代码块没有被执行：这是因为第一个代码块并没有返回错误。
让我们在``try``代码块中放一个有问题的语句看看会发生什么：

In [8]:
try:
    print("let's try something:")
    x = 1 / 0 # ZeroDivisionError
except:
    print("something bad happened!")

let's try something:
something bad happened!


这里我们可以看到，当一个错误（这里是一个``ZeroDivisionError``）在``try``语句中被触发时，这个错误被捕获，然后``except``语句会被执行。

这是我们在一个函数或者其他代码中检测用户输入的一种方法。
比如，我们可能期望有一个函数可以捕获一个除零异常并返回一个其他值，也许是一个很大的数字，比如$10^{100}$:

In [9]:
def safe_divide(a, b):
    try:
        return a / b
    except:
        return 1E100

In [10]:
safe_divide(1, 2)

0.5

In [11]:
safe_divide(2, 0)

1e+100

这里有一个细微的问题，如果发生了其他的异常会出现什么状况，比如这里有一个不符合我们预期的例子：

In [12]:
safe_divide (1, '2')

1e+100

在整数和字符串之间运用除法会触发一个``TypeError``异常， 但是我们上边写的那段过于热情的代码把它当做``ZeroDivisionError``来处理了。
因此，最好要明确指出我们要捕获的异常类型：

In [13]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100

In [14]:
safe_divide(1, 0)

1e+100

In [15]:
safe_divide(1, '2')

TypeError: unsupported operand type(s) for /: 'int' and 'str'

现在我们只能捕获除零异常，让其他异常都原封不动地传出去。

## 抛出异常： ``raise``
我们已经看到了在使用 Python 时，能获得异常的信息的重要性。

在你编写的代码中使用具有信息的异常同样有价值，因为这样你的代码的用户（最重要的是你自己！）可以弄清导致错误的原因。

你来抛出自己的异常的方法是使用``raise``语句，比如：

In [16]:
raise RuntimeError("my error message")

RuntimeError: my error message

这里使用这个例子很合适，让我们回到我们前边定义的``fibonacci``函数：

In [17]:
def fibonacci(N):
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

这里有一个潜在的问题是，输入的值可能为负值。
这并不会让我们的函数现在出现什么错误，但是我们可能希望让用户知道一个负的``N``是不支持的。

根据惯例，由于无效参数值导致的错误导致``ValueError``：

In [18]:
def fibonacci(N):
    if N < 0:
        raise ValueError("N must be non-negative")
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [19]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [20]:
fibonacci(-10)

ValueError: N must be non-negative

现在用户能够确切地知道为什么输入是无效的了，然后可以用 ``try``...``except``来处理它。

In [21]:
N = -10
try:
    print("trying this...")
    print(fibonacci(N))
except ValueError:
    print("Bad value: need to do something else")

trying this...
Bad value: need to do something else


## 深入探究异常

简单地说，我想要在这提一些你可能会遇到的概念。
我不会详细介绍这些概念，以及如何和为什么使用它们，而只是向你展示语法，之后你可以自己探索更多关于它们的内容。

### 访问错误信息Accessing the error message

有时在``try``...``except``语句中，你可能会想要处理一下错误信息。
这是你可以使用``as``：

In [22]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("Error class is:  ", type(err))
    print("Error message is:", err)

Error class is:   <class 'ZeroDivisionError'>
Error message is: division by zero


通过这种方法，你可以进一步自定义函数的异常处理。

### 定义自定义异常
除了内置异常之外，还可以通过类继承来定义自定义异常。
例如，如果你想要一个特殊的``ValueError``，你可以这样做：

In [23]:
class MySpecialError(ValueError):
    pass

raise MySpecialError("here's the message")

MySpecialError: here's the message

这将允许你使用一个只能捕获此类错误的``try`` ...``except``块。

In [24]:
try:
    print("do something")
    raise MySpecialError("[informative error message here]")
except MySpecialError:
    print("do something else")

do something
do something else


在开发更多自定义代码时，你可能会发现这很有用。

## ``try``...``except``...``else``...``finally``
除了``try``和``except``之外，你可以使用``else``和``finally``进一步调整代码的异常处理。
基本结构是这样的：

In [1]:
try:
    print("try something here")
except:
    print("this happens only if it fails")
else:
    print("this happens only if it succeeds")
finally:
    print("this happens no matter what")

try something here
this happens only if it succeeds
this happens no matter what


这里的``else``这个功能很清楚，但是``finally``的意思是什么？
其实无论发生什么,``finally``子句始终会被执行，我通常认为它是在操作完成后用来做某种清理工作的。

<!--NAVIGATION-->
< [Defining and Using Functions](08-Defining-Functions.ipynb) | [Contents](Index.ipynb) | [Iterators](10-Iterators.ipynb) >